In [1]:
import pandas as pd
raw = pd.read_csv('/Users/albertsun/Projects/data/job_access_gap.csv')
urban_institute = raw.copy()


In [2]:
urban_institute['GEOID'] = urban_institute['GEOID'].astype('str')

In [3]:
"""
p_matchings_path = '/Users/albertsun/Projects/data/geoid_high_poverty.csv'
p_matchings = pd.read_csv(p_matchings_path)
p_matchings
urban_institute_m = pd.merge(urban_institute, p_matchings)
urban_institute_m
"""

"\np_matchings_path = '/Users/albertsun/Projects/data/geoid_high_poverty.csv'\np_matchings = pd.read_csv(p_matchings_path)\np_matchings\nurban_institute_m = pd.merge(urban_institute, p_matchings)\nurban_institute_m\n"

In [4]:
urban_institute.describe()

,OverallJobAccess,lowwagelaborforce,Access30Transit,Access30Transit_ts,spatialmismatch
count,5823.000000,5823.000000,5823.000000,5823.000000,5823.000000
mean,0.985180,380.676455,0.094956,0.052541,0.086366
std,0.578901,211.519522,0.192307,0.108627,0.053932
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.562588,247.000000,0.003566,0.001999,0.051455
50%,0.869975,337.000000,0.020009,0.010281,0.075283
75%,1.299884,461.000000,0.084798,0.049593,0.108407
max,3.632130,3243.000000,3.046875,3.046875,0.818570


In [5]:
urban_institute = urban_institute[urban_institute.MSA == "Seattle"]

In [6]:
def discretize(columns, df):
    for column in columns: 
        df[column + '_D'] = pd.qcut(df[column], q=4, labels=[1, 2, 3, 4])
        df.drop(column, axis=1)
    return df

columns = ['OverallJobAccess', 'lowwagelaborforce', 'Access30Transit', 'Access30Transit_ts', 'spatialmismatch']
urban_institute = discretize(columns, urban_institute)
columns_d = [i + '_D' for i in columns]

In [7]:
urban_institute = urban_institute[['GEOID'] + columns_d]
urban_institute

,GEOID,OverallJobAccess_D,lowwagelaborforce_D,Access30Transit_D,Access30Transit_ts_D,spatialmismatch_D
0,530610518023,4,3,3,3,3
1,530530634006,3,1,2,1,1
2,530330219042,4,1,3,4,1
3,530330219044,3,1,3,4,1
4,530330219051,4,2,3,3,1
...,...,...,...,...,...,...
2478,530330293035,3,3,1,1,3
2479,530330300046,3,4,3,3,4
2480,530330303081,1,2,2,2,2
2481,530330323272,2,1,2,2,1


In [8]:
reg_path = '/Users/albertsun/Projects/data/LIFT_registry_2022-03-22.csv'
reg = pd.read_csv(reg_path)
reg['FIPS'] = reg['FIPS'].astype(str)
reg['FIPS'] = reg['FIPS'].str[:12]
print(len(set(reg['FIPS'])), len(set(urban_institute['GEOID'])), len(set(reg['FIPS']).intersection(set(urban_institute['GEOID']))))
reg = reg.rename(columns={'FIPS': 'GEOID'})
reg


2531 2483 2392


,Age,RaceDesc,LanguageSpoken,Expiration,DateIssued,CardIssuingAgency,GEOID,Initial Load,Study Card,card_id
0,36,Black or African American,Other,8/31/2019,2017-06-15,KCMCCS,530330288022,NaN,NaN,568-1
1,56,White,English,7/31/2019,2017-06-05,YWCA,530330040001,NaN,NaN,2690-1
2,58,White,English,11/30/2021,2019-08-28,King County Public Health,530330040001,NaN,NaN,2690-2
3,49,Black or African American,English,11/30/2023,2022-01-26,DSHS - ORCA LIFT (remote enrollment),530530609051,10.0,NaN,2704-1
4,46,Asian,English,5/31/2021,2019-06-30,King County Public Health,530330061002,NaN,NaN,2706-1
...,...,...,...,...,...,...,...,...,...,...
117723,26,White,English,3/31/2025,2022-03-17,King County Public Health,530330303103,10.0,NaN,272812-1
117724,26,Black or African American,English,3/31/2025,2022-03-17,King County Public Health,530330004011,10.0,NaN,272814-1
117725,52,White,English,10/31/2024,2022-03-17,King County Public Health,530330296011,NaN,Subsidized Annual Pass,272816-1
117726,58,Black or African American,English,3/31/2025,2022-03-17,King County Public Health,530330289021,10.0,NaN,272818-1


In [9]:
reg[reg['Study Card'] == 'Subsidized Annual Pass']

,Age,RaceDesc,LanguageSpoken,Expiration,DateIssued,CardIssuingAgency,GEOID,Initial Load,Study Card,card_id
53,52,Hispanic or Latino,English,10/31/2023,2020-12-09,King County Public Health,530330207002,NaN,Subsidized Annual Pass,2824-1
54,52,Hispanic or Latino,English,3/31/2022,2020-12-09,King County Public Health,530330207002,NaN,Subsidized Annual Pass,2824-2
55,53,Hispanic or Latino,English,10/31/2023,2021-11-17,King County Public Health,530330207002,NaN,Subsidized Annual Pass,2824-3
75,67,White,English,8/31/2024,2021-04-20,DSHS - subsidized annual pass (remote enrollment),530330090001,NaN,Subsidized Annual Pass,2878-1
80,69,White,English,7/31/2023,2020-11-16,DSHS - subsidized annual pass (remote enrollment),530330303141,NaN,Subsidized Annual Pass,2898-1
...,...,...,...,...,...,...,...,...,...,...
117711,40,Black or African American,English,1/31/2024,2022-03-17,DSHS - subsidized annual pass (remote enrollment),530330085002,NaN,Subsidized Annual Pass,272788-1
117712,41,Black or African American,English,10/31/2024,2022-03-17,King County Public Health,530330297005,NaN,Subsidized Annual Pass,272790-1
117713,12,Black or African American,English,1/31/2024,2022-03-17,DSHS - subsidized annual pass (remote enrollment),530330085002,NaN,Subsidized Annual Pass,272792-1
117714,23,White,English,1/31/2024,2022-03-17,DSHS - subsidized annual pass (remote enrollment),530610536023,NaN,Subsidized Annual Pass,272794-1


In [14]:
merged = pd.merge(reg, urban_institute, on='GEOID', how='inner')
data_path = '/Users/albertsun/Projects/data/'
merged.to_csv(data_path + 'LIFT_registry__Merged.csv')

In [11]:
merged[merged['Study Card'] == 'Subsidized Annual Pass']

,Age,RaceDesc,LanguageSpoken,Expiration,DateIssued,CardIssuingAgency,GEOID,Initial Load,Study Card,card_id,OverallJobAccess_D,lowwagelaborforce_D,Access30Transit_D,Access30Transit_ts_D,spatialmismatch_D
82,34,Hispanic or Latino,English,7/31/2023,2021-01-05,DSHS - subsidized annual pass (remote enrollment),530330288022,NaN,Subsidized Annual Pass,243548-1,4,3,1,1,2
85,50,White,English,8/31/2024,2021-11-02,King County Public Health,530330288022,NaN,Subsidized Annual Pass,263692-1,4,3,1,1,2
89,46,Black or African American,English,8/31/2023,2020-10-26,King County Public Health,530330040001,NaN,Subsidized Annual Pass,3602-2,1,2,4,4,3
90,47,Black or African American,English,8/31/2023,2021-11-01,King County Public Health,530330040001,NaN,Subsidized Annual Pass,3602-3,1,2,4,4,3
95,60,Multi-Racial,English,10/31/2023,2021-02-18,CCS,530330040001,NaN,Subsidized Annual Pass,4218-2,1,2,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112507,60,White,English,7/31/2024,2021-06-22,CCS,530330327023,NaN,Subsidized Annual Pass,251106-2,2,1,2,2,2
112511,42,White,English,8/31/2024,2021-05-25,DSHS - subsidized annual pass (remote enrollment),530330328002,NaN,Subsidized Annual Pass,253006-1,1,2,2,3,3
112512,20,White,English,8/31/2024,2021-05-25,DSHS - subsidized annual pass (remote enrollment),530330328002,NaN,Subsidized Annual Pass,253008-1,1,2,2,3,3
112520,39,White,English,10/31/2024,2022-02-14,King County Public Health,530530723056,NaN,Subsidized Annual Pass,270140-1,2,1,1,1,1


In [12]:
(11666 - 11443) / 11666 * 100 

1.9115378021601233

Only about 2 percent of the people with SAP are removed 

OverallJobAccess - 
lowwagelaborforce
Access30Transit
Access30Transit_ts
spatialmismatch
